In [1]:
from nikkos_tools.cloudy_tools import cloudy_input_builder as cib
from nikkos_tools import miscellaneous_functions as mf
import numpy as np  
import itertools
import os
MODEL_PATH = mf.make_directories('./models/v0.1')

Folder ./models/v0.1 already exists


In [2]:
gas_metallicities = [0.00001]
stellar_metallicities = np.round(np.log10(gas_metallicities), decimals=2)
ages = [1e6]

for i, j in enumerate(itertools.product(gas_metallicities, ages)):
    z_gas = j[0]
    z_star = np.log10(z_gas)
    age = j[1]
    stars = cib.CloudyModel()
    stars.build_template_model_bpass(gas_metallicity=z_gas, stellar_metallicity=z_star, age=age)
    stars.make_cloudy_in_file(path=MODEL_PATH)
    
cib.make_cloudy_executable(MODEL_PATH, 'cloudytest_v0')

In [7]:
# gas_metallicities = [0.00001, 0.0001, 0.005, 0.01, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.7, 1, 1.5, 2]
# stellar_metallicities = np.round(np.log10(gas_metallicities), decimals=2)
# ages = [1e6, 3e6, 5e6, 1e7, 1e8, 1e9, 1e10]

# for i, j in enumerate(itertools.product(gas_metallicities, ages)):
#     z_gas = j[0]
#     z_star = np.log10(z_gas)
#     age = j[1]
#     stars = cib.CloudyModel()
#     stars.build_template_model_bpass(gas_metallicity=z_gas, stellar_metallicity=z_star, age=age)
#     stars.make_cloudy_in_file(path=MODEL_PATH)